<a href="https://colab.research.google.com/github/Sooraj9503/cuda/blob/main/matrixMultiplicationOnGpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%writefile matrixMul.cu

#include<iostream>
#include<cstdlib>
#include<math.h>
#include<iomanip>
#include<cuda_runtime.h>
using namespace std;

__global__ void mul(int *arr1, int *arr2, int *arr3, int rows, int cols)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if(tid < rows * cols)
    {
        arr3[tid] = arr1[tid] * arr2[tid];
    }
}

int main()
{
    int rows = 5, cols = 5;
    int *harr1, *harr2, *harr3, *darr1, *darr2, *darr3;
    size_t size = rows * cols * sizeof(int);

    harr1 = (int*)malloc(size);
    harr2 = (int*)malloc(size);
    harr3 = (int*)malloc(size);

    cudaMalloc(&darr1, size);
    cudaMalloc(&darr2, size);
    cudaMalloc(&darr3, size);

    for(int i = 0; i < rows; i++)
    {
        for(int j = 0; j <cols; j++)
        {
            harr1[i * cols + j] = rand() % 5;
            harr2[i * cols + j] = rand() % 5;
        }
    }

    cudaMemcpy(darr1, harr1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(darr2, harr2, size, cudaMemcpyHostToDevice);

    int threadPerBlock = 256;
    int blocks = (rows * cols + threadPerBlock - 1) / threadPerBlock;

    mul<<<blocks, threadPerBlock>>>(darr1, darr2, darr3, rows, cols);
    cudaDeviceSynchronize();

    cudaMemcpy(harr3, darr3, size, cudaMemcpyDeviceToHost);

    for(int i = 0; i < rows; i++)
    {
        for(int j = 0; j < cols; j++)
        {
            cout<<setw(4)<<harr3[i * cols +j];
        }
        cout<<endl;
    }

    cudaFree(darr1);
    cudaFree(darr2);
    cudaFree(darr3);
    free(harr1);
    free(harr2);
    free(harr3);

    return 0;
}

Overwriting matrixMul.cu


In [14]:
!nvcc matrixMul.cu

In [15]:
!./a.out

   3   0   0   2   4
   4   0   3   0   2
   3   8   0   6   0
   8   6   8   3   2
  12   4   8  12   0
